## Using the `%trace` magic command to visualize your Q# operations

In quantum computing literature, it is common to visualize quantum algorithms as quantum circuits. For example, when learning about [Simon's algorithm](https://en.wikipedia.org/wiki/Simon%27s_problem#Simon's_quantum_circuit) or [Quantum Teleportation](https://techcommunity.microsoft.com/t5/educator-developer-blog/quantum-teleportation-in-q/ba-p/380602) for the first time, it is useful to visualize them as a quantum circuit. However, there are limitations to this approach which will be discussed [below](#Limitations-of-a-quantum-circuit).

The `%trace` command allows developers to debug their Q# programs by visualizing an execution path that their program takes. It "traces" through the execution path and keeps track of the operations passed into the simulator for visualization.

To see a description of what the `%trace` command does, we can run `%trace?`:

In [1]:
%trace?

%trace:
Visualizes the execution path of the given operation.

### Basic operations
To start off, let's see how we can debug basic operations with the `%trace` magic command. First, let's define an operation `RunBasicOp` that will perform an `H` on the first qubit and an `X` on the second.

In [2]:
operation RunBasicOp() : Unit {
    using (qs = Qubit[2]) {
        H(qs[0]);
        X(qs[1]);
        ResetAll(qs);
    }
}

RunBasicOp

Now, we use the `%trace` magic command to render a visualization of `RunBasicOp` in the cell.

In [3]:
%trace RunBasicOp

Microsoft.Quantum.IQSharp.Jupyter.DisplayableHtmlElement

We can extend this to visualize useful quantum algorithms, such as the Deutsch–Jozsa algorithm. The following operation can be found in our Q# samples [here](https://github.com/microsoft/Quantum/blob/master/samples/getting-started/simple-algorithms/SimpleAlgorithms.qs).

In [4]:
open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Arrays;

operation ApplyOracle (query: Qubit[], target: Qubit) : Unit {
    // ...
}

operation IsConstantBooleanFunction (queryRegister: Qubit[], target: Qubit, n : Int) : Bool {
    X(target);
    H(target);

    within {
        ApplyToEachA(H, queryRegister);
    } apply {
        // |𝑥, 𝑦〉 ↦ |𝑥, 𝑦 ⊕ 𝑓(𝑥)〉.
        ApplyOracle(queryRegister, target);
    }

    let resultArray = ForEach(MResetZ, queryRegister);

    return All(IsResultZero, resultArray);
}

operation RunDeutschJozsa (n : Int) : Bool {
    using ((queryRegister, target) = (Qubit[n], Qubit())) {
        let res = IsConstantBooleanFunction(queryRegister, target, n);
        Reset(target);
        return res;
    }
}

ApplyOracle, IsConstantBooleanFunction, RunDeutschJozsa

In [5]:
%trace RunDeutschJozsa n=3

Microsoft.Quantum.IQSharp.Jupyter.DisplayableHtmlElement

We passed in an extra argument `n=3` to the `%trace` command. Any argument that needs to be passed in to the given Q# operation is denoted by a key-value pair `<param>=<value>`. More details can be found in the documentation by running `%trace?`.

Note that the visualization renders `IsConstantBooleanFunction` as a giant gate rather than its components. This is because, by default, the depth at which operations are traced out is at depth `1`. The default depth can be changed by running the magic command `%config trace.defaultDepth=n`, where `n` is the desired default depth.

To see the internals of `IsConstantBooleanFunction`, we need to render operations up to 2 levels deep by passing in the `--depth=2` flag. Try changing the value of the depth and see how the visualization changes!

In [6]:
%trace RunDeutschJozsa n=3 --depth=2

Microsoft.Quantum.IQSharp.Jupyter.DisplayableHtmlElement

### Limitations of a quantum circuit
Quantum circuits are a great way of thinking about quantum algorithms that are fairly simple, but it does not scale well to arbitrary quantum programs. As quantum computing and quantum programming matures, we will see classical computer science paradigms, such as if statements and while loops, appear in our quantum programs (check out how Q# accomplishes this [here](https://docs.microsoft.com/en-us/quantum/user-guide/using-qsharp/control-flow)).

As more control branches are created, it quickly becomes infeasible to visualize a quantum program as simple quantum circuits. Instead, we modify the quantum circuit to incorporate these classical paradigms by tracing through and visualizing a given runtime execution path.

### Non-deterministic Q# programs

Notice that, so far, the visualizations are exactly the quantum circuits that one would normally visualize them as. This is because the past examples are deterministic Q# programs. An example of a non-deterministic algorithm is this solution for one of the Q# coding contest questions hosted in [Summer 2020](https://codeforces.com/blog/entry/77614):

You are given N qubits in the state $|0\ldots0\rangle$. Your task is to prepare an equal superposition of all basis states that have one or more 0 in them.

For example, for $N=2$ the required state would be $\frac{1}{\sqrt{3}}(|00\rangle+|01\rangle+|10\rangle)$.

In [7]:
// Sets all zero state to an equal superposition of all states except the |1..1> state
operation SetEqualSuperposition (qs : Qubit[]) : Unit {
    using (anc = Qubit()) { 
        repeat {
            ApplyToEach(H, qs);
            Controlled X(qs, anc);
            let res = M(anc);
            Reset(anc);
        } 
        until (res == Zero)
        fixup {
            ResetAll(qs);
        }
    }
}

operation RunSolution () : Unit {
    using (qs = Qubit[2]) {
        SetEqualSuperposition(qs);
        ResetAll(qs);
    }
}

SetEqualSuperposition, RunSolution

In [8]:
%trace RunSolution --depth=2

Microsoft.Quantum.IQSharp.Jupyter.DisplayableHtmlElement

Try running the above cell multiple times. You'll see that the visualization isn't always the same! This is because we're visualizing an _execution path_ of a non-deterministic algorithm as opposed to a deterministic quantum circuit. The process of repeatedly performing operations until we get a desired measurement is non-deterministic and, thus, our `%trace` operation treats it as such.

### Classically-controlled operations
Future improvements to this magic command includes the ability to visualize classically-controlled operations. A simple algorithm that includes operations conditioned on measurements is quantum teleportation. The following operation can be found in our Q# samples [here](https://github.com/microsoft/Quantum/blob/master/samples/getting-started/teleportation/TeleportationSample.qs).

In [9]:
open Microsoft.Quantum.Measurement;

operation Teleport (msg : Qubit, target : Qubit) : Unit {
    using (register = Qubit()) {
        // Create some entanglement that we can use to send our message.
        H(register);
        CNOT(register, target);

        // Encode the message into the entangled pair.
        CNOT(msg, register);
        H(msg);

        if (MResetZ(msg) == One) { Z(target); }
        
        if (IsResultOne(MResetZ(register))) { X(target); }
    }
}

operation TeleportState () : Unit {
    using ((msg, target) = (Qubit(), Qubit())) {
        Ry(1.5, msg);
        Teleport(msg, target);
        Reset(msg);
        Reset(target);
    }
}

Teleport, TeleportState

In [10]:
%trace TeleportState --depth=2

Microsoft.Quantum.IQSharp.Jupyter.DisplayableHtmlElement

In this case, we have operations that are only executed based on a given measurement outcome. Currently, we render a fixed execution branch traced out by the simulator. Future work will allow toggling between each measurement result (unset, 1, 0) and renders the operations in the corresponding execution branch. This will allow users to visualize how their program evolves depending on the measurement result.

Of course, quantum teleportation is a fairly simple example which can still be represented as a quantum circuit using the [Principle of Deferred Measurement](https://en.wikipedia.org/wiki/Deferred_Measurement_Principle) but we can appreciate how this would be useful as quantum programs become larger and more complex.

### Version information
This notebook was last executed with:

In [11]:
%version

Component,Version
iqsharp,0.12.2007.3110
Jupyter Core,1.4.0.0
.NET Runtime,".NETCoreApp,Version=v3.1"
